In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         os.path.join(dirname, filename)
# print("Path Join Completed")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install PyWavelets
!pip install kaggle --upgrade
from keras.models import Model
from keras.layers import Input, GlobalAveragePooling2D, Dense
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras.callbacks import EarlyStopping
import pandas as pd
import numpy as np
from keras.regularizers import l2
from tensorflow.keras.applications import InceptionV3
import cv2
from skimage.filters import threshold_otsu
from skimage.restoration import wiener
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from skimage import exposure
from skimage.restoration import wiener
from skimage.morphology import disk
import pywt
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import seaborn as sns

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.2/181.2 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.7.4.2
    Uninstalling kaggle-1.7.4.2:
      Successfully uninstalled kaggle-1.7.4.2


In [16]:
import kagglehub
import pandas as pd
import os
import shutil

# Step 1: Download the dataset
path = kagglehub.dataset_download("awsaf49/cbis-ddsm-breast-cancer-image-dataset")
print("Path to dataset files:", path)

# Step 2: Find CSV file
csv_path = None
for root, dirs, files in os.walk(path):
    for file in files:
        if file.endswith(".csv"):
            csv_path = os.path.join(root, file)
            print("Found CSV file:", csv_path)

# Step 3: Read the CSV (optional)
if csv_path:
    df = pd.read_csv(csv_path)
    print("First 5 rows of the dataset:")
    print(df.head())
else:
    print("No CSV file found in the dataset.")

# Step 4: Prepare folders
has_cancer_dir = os.path.join("sorted_ddsm_images", "has_cancer")
no_cancer_dir = os.path.join("sorted_ddsm_images", "no_cancer")

os.makedirs(has_cancer_dir, exist_ok=True)
os.makedirs(no_cancer_dir, exist_ok=True)

# Step 5: Sort and save images
image_extensions = (".png", ".jpg", ".jpeg", ".tif", ".tiff")
has_count = 0
no_count = 0

for root, dirs, files in os.walk(path):
    for file in files:
        if file.lower().endswith(image_extensions):
            original_path = os.path.join(root, file)
            first_char = os.path.basename(file)[0]

            if first_char == "1":
                target_path = os.path.join(has_cancer_dir, file)
                has_count += 1
            elif first_char == "2":
                target_path = os.path.join(no_cancer_dir, file)
                no_count += 1
            else:
                continue  # skip files that don't match 1 or 2

            if not os.path.exists(target_path):
                shutil.copy2(original_path, target_path)

print(f"Saved {has_count} cancer-positive images to: {os.path.abspath(has_cancer_dir)}")
print(f"Saved {no_count} cancer-negative images to: {os.path.abspath(no_cancer_dir)}")


Path to dataset files: /kaggle/input/cbis-ddsm-breast-cancer-image-dataset
Found CSV file: /kaggle/input/cbis-ddsm-breast-cancer-image-dataset/csv/dicom_info.csv
Found CSV file: /kaggle/input/cbis-ddsm-breast-cancer-image-dataset/csv/mass_case_description_train_set.csv
Found CSV file: /kaggle/input/cbis-ddsm-breast-cancer-image-dataset/csv/calc_case_description_train_set.csv
Found CSV file: /kaggle/input/cbis-ddsm-breast-cancer-image-dataset/csv/meta.csv
Found CSV file: /kaggle/input/cbis-ddsm-breast-cancer-image-dataset/csv/calc_case_description_test_set.csv
Found CSV file: /kaggle/input/cbis-ddsm-breast-cancer-image-dataset/csv/mass_case_description_test_set.csv
First 5 rows of the dataset:
  patient_id  breast_density left or right breast image view  abnormality id  \
0    P_00016               4                 LEFT         CC               1   
1    P_00016               4                 LEFT        MLO               1   
2    P_00017               2                 LEFT         

In [17]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
import os

# Set paths
data_dir = "sorted_ddsm_images"
img_size = (224, 224)
batch_size = 32

# Step 1: Create ImageDataGenerators
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2,
    horizontal_flip=True,
    rotation_range=20,
    zoom_range=0.2,
    shear_range=0.2
)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training',
    shuffle=True
)

val_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation',
    shuffle=False
)

# Step 2: Load VGG19 base model
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze base layers

# Step 3: Build the model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')  # Binary classification
])

# Step 4: Compile the model
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Step 5: Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10
)

# Step 6: Evaluate on validation set
val_loss, val_acc = model.evaluate(val_generator)
print(f"Validation Accuracy: {val_acc:.2f}")


Found 480 images belonging to 2 classes.
Found 120 images belonging to 2 classes.
80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 61s 3s/step - accuracy: 0.5579 - loss: 1.1317 - val_accuracy: 0.6000 - val_loss: 0.6417
Epoch 2/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 34s 2s/step - accuracy: 0.6215 - loss: 0.8025 - val_accuracy: 0.7667 - val_loss: 0.4924
Epoch 3/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - accuracy: 0.6783 - loss: 0.6944 - val_accuracy: 0.8667 - val_loss: 0.4072
Epoch 4/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 36s 2s/step - accuracy: 0.7285 - loss: 0.5968 - val_accuracy: 0.8750 - val_loss: 0.3588
Epoch 5/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 38s 3s/step - accuracy: 0.7512 - loss: 0.6516 - val_accuracy: 0.8750 - val_loss: 0.3306
Epoch 6/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 34s 2s/step - accuracy: 0.7513 - loss: 0.6483 - val_accuracy: 0.8750 - val_loss: 0.3352
Epoch 7/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - accuracy: 0.7788 - loss: 0.4939 - val_accuracy: 0.9083 - val_loss: 0.2999
Epoch 8/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 35s 2s/step - accuracy: 0.7614 - loss: 0.5986 - val_accuracy: 0.8917 - val_loss:

In [12]:
df.head()

,SeriesInstanceUID,StudyInstanceUID,Modality,SeriesDescription,BodyPartExamined,SeriesNumber,Collection,Visibility,ImageCount
0,1.3.6.1.4.1.9590.100.1.2.117041576511324414842...,1.3.6.1.4.1.9590.100.1.2.229361142710768138411...,MG,ROI mask images,BREAST,1,CBIS-DDSM,1,2
1,1.3.6.1.4.1.9590.100.1.2.438738396107617880132...,1.3.6.1.4.1.9590.100.1.2.195593486612988388325...,MG,ROI mask images,BREAST,1,CBIS-DDSM,1,2
2,1.3.6.1.4.1.9590.100.1.2.767416741131676463382...,1.3.6.1.4.1.9590.100.1.2.257901172612530623323...,MG,ROI mask images,BREAST,1,CBIS-DDSM,1,2
3,1.3.6.1.4.1.9590.100.1.2.296931352612305599800...,1.3.6.1.4.1.9590.100.1.2.109468616710242115222...,MG,ROI mask images,BREAST,1,CBIS-DDSM,1,2
4,1.3.6.1.4.1.9590.100.1.2.436657670120353100077...,1.3.6.1.4.1.9590.100.1.2.380627129513562450304...,MG,ROI mask images,BREAST,1,CBIS-DDSM,1,2


In [14]:
df.shape

(6775, 9)